In [1]:
import psycopg2
import csv

In [2]:
db = psycopg2.connect(
    user="lvs215",
    password="",
    host="localhost",
    port="5432",
    database="lvs215"
)

In [9]:
queries = [
    {
        "name": "reproducibility",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-system",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%') 
                AND
                (lower(ps.title) LIKE '%system%' OR lower(ps.abstract) LIKE '%system%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-systems",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%') 
                AND
                (lower(ps.title) LIKE '%systems%' OR lower(ps.abstract) LIKE '%systems%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-or-repeatability",
        "query": """
                SELECT ps.id, ps.title, ps.abstract, ps.year, ps.venue, ps.doi, ps.n_citations
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                )
                ORDER BY ps.id
                """,
        "run": False
    },
    {
        "name": "reproducibility-or-repeatability-or-replicability",
        "query": """
                SELECT ps.id, ps.title, ps.abstract, ps.year, ps.venue, ps.doi, ps.n_citations
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                    OR
                    (lower(ps.title) LIKE '%replicability%' OR lower(ps.abstract) LIKE '%replicability%')
                )
                ORDER BY ps.id
                """,
        "run": False
    },
    {
        "name": "reproducibility-or-repeatability-or-replicability-v2",
        "query": """
                SELECT ps.id, ps.title, ps.abstract, ps.year, ps.venue, ps.doi, ps.n_citations, string_agg(a.name, ', ') as authors
                FROM publications ps 
                    LEFT JOIN author_paper_pairs app ON CAST(ps.id AS VARCHAR) = app.paper_id
                    LEFT JOIN authors a ON CAST(a.id AS VARCHAR) = app.author_id
                WHERE 
                ( 
                    (lower(ps.title) ~ 'reproducibility' OR lower(ps.abstract) ~ 'reproducibility')
                    OR
                    (lower(ps.title) ~ 'repeatability' OR lower(ps.abstract) ~ 'repeatability')
                    OR
                    (lower(ps.title) ~ 'replicability' OR lower(ps.abstract) ~ 'replicability')
                )
                GROUP BY ps.id, ps.title, ps.abstract, ps.year, ps.venue, ps.doi, ps.n_citations
                ORDER BY ps.id
                """,
        "run": True
    },
    {
        "name": "reproducibility-or-repeatability-system",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                )
                AND
                (lower(ps.title) LIKE '%system%' OR lower(ps.abstract) LIKE '%system%') 
                """,
        "run": False
    },
    {
        "name": "reproducibility-or-repeatability-systems",
        "query": """
                SELECT ps.id, ps.title 
                FROM publications ps
                WHERE 
                ( 
                    (lower(ps.title) LIKE '%reproducibility%' OR lower(ps.abstract) LIKE '%reproducibility%')
                    OR
                    (lower(ps.title) LIKE '%repeatability%' OR lower(ps.abstract) LIKE '%repeatability%')
                )
                AND
                (lower(ps.title) LIKE '%systems%' OR lower(ps.abstract) LIKE '%systems%') 
                """,
        "run": False
    }
]

In [10]:
def test_query(query_dict):
    print(query_dict['name'])
    query_results = []
    if not query_dict['run']:
        print('skipped')
        return
    with db.cursor() as cursor:
        cursor.execute(query_dict["query"])
        field_names = [field[0] for field in cursor.description]
        query_results = cursor.fetchall()
    print("General Statistics:", "\n", "result length", len(query_results), "\n")
    with open(f"{query_dict['name']}.csv", 'w+', newline ='') as file:
        write = csv.writer(file)
        write.writerow(field_names)
        write.writerows(query_results)
    for pub in query_results[1:10]:
        print(pub)
    

In [11]:
for q in queries:
    print("#"*80, "\n")
    test_query(q)
    print("\n")

################################################################################ 

reproducibility
skipped


################################################################################ 

reproducibility-system
skipped


################################################################################ 

reproducibility-systems
skipped


################################################################################ 

reproducibility-or-repeatability
skipped


################################################################################ 

reproducibility-or-repeatability-or-replicability
skipped


################################################################################ 

reproducibility-or-repeatability-or-replicability-v2
General Statistics: 
 result length 151 

('13a572f4922a195882e6e1a1ddc5c2fabddde7c8', 'Research Software Discovery: An Overview', 'Research software is an integral part of scientific investigations. The paper identifies challenges, risks and new opport